In [1]:
import fastf1
from fastf1 import api
import pandas as pd
from matplotlib import pyplot as plt
from dfply import *
from datetime import datetime
from datetime import timedelta
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#note: you should put in a new folder to store cached data, github doesn't push empty folders so you have to do it manually
#In the zip, FastF1Cache should have data in it so it loads faster
fastf1.Cache.enable_cache('FastF1Cache')

# Data Scraping

In [2]:
def QualiScrape(grandPrix, raceDate, qualiDate):
    """
    Input: grandPrix, Grand Prix's Race Date, Date of Qualifying
    Returns: data frame with each driver matched to best qualifying time, along with tire and weather conditions
    """
    #Build path to scrape data from
    Quali = fastf1.api.make_path(grandPrix, raceDate, 'Qualifying', qualiDate)
    
    #Scraping timing data, timing app data (for info on tire compund), and weather data
    QualiTiming = fastf1.api.timing_data(Quali)[0]
    QualiTimingApp = fastf1.api.timing_app_data(Quali)
    QualiWeatherDict = fastf1.api.weather_data(Quali)
    QualiWeatherData = pd.DataFrame.from_dict(QualiWeatherDict)
    
    #Remove entries with no laptimes, and round session time to nearest minute (for merging purposes)
    QualiTiming = QualiTiming[QualiTiming.LapTime.notnull()]
    QualiTimingApp >>= mutate(Time = X.Time.round('60s')) >> arrange(X.Driver, X.Time)
    QualiTimingApp = QualiTimingApp.reset_index()
    
    #In timing app dataframe, tire compound is only noted when new set is put on
    #Since dataframe is organized by driver and then time, we can replace empty entries with the previous tire compound
    #Also, dataframe sometime doubles on driver-time combo, so removing the unnecessary ones
    for index in range(len(QualiTimingApp)):
        if not QualiTimingApp.loc[index,"Compound"]:
            QualiTimingApp.loc[index, "Compound"] = QualiTimingApp.loc[index-1, "Compound"]
        if not QualiTimingApp.loc[index,"New"]:
            QualiTimingApp.loc[index,"New"] = False
        if index == 0:
            continue
        if QualiTimingApp.loc[index,"Time"] == QualiTimingApp.loc[index-1,"Time"]:
            if not QualiTimingApp.loc[index,"LapTime"]:
                QualiTimingApp.loc[index,"Driver"] = np.nan
            else: 
                QualiTimingApp.loc[index-1,"Driver"] = np.nan                
    QualiTimingApp = QualiTimingApp[QualiTimingApp.Driver.notnull()]
    
    #Selecting relevant rows
    QualiTimingApp >>= select(X.Stint, X.Driver, X.TotalLaps, X.Compound, X.New, X.Time)
    QualiTiming = QualiTiming >> mutate(Time = X.Time.round('60s')) >> arrange(X.Driver, X.Time) >> \
              drop(contains("Session"), X.PitOutTime, X.PitInTime)

    QualiCompiledTiming = pd.merge(QualiTiming,QualiTimingApp, on=['Time', 'Driver'])

    QualiWeatherData >>= mutate(Time = X.Time.round('60s'))
    QualiCompiledTiming = pd.merge(QualiCompiledTiming, QualiWeatherData, on=['Time'])
    QualiCompiledTiming >>= arrange(X.Driver, X.Time)
    QualiCompiledTiming = QualiCompiledTiming.reset_index()
    
    #Creating List of all drivers, then finding the best lap time for them
    driverList = QualiCompiledTiming.Driver.unique()
    QualiTimes = {}
    for drv in driverList:
        QualiTimes[drv] = [drv, QualiTiming[QualiTiming.Driver == drv].LapTime.min()]
    QualiDF = pd.DataFrame.from_dict(QualiTimes, orient = 'index', columns = ['Driver','LapTime'])
    QualiDF = QualiDF.reset_index()
    QualiDF >>= select(X.Driver, X.LapTime)
    
    #Merge weather and tire data with appropriate laptime data
    QualiDF = pd.merge(QualiDF, QualiCompiledTiming, on = ["LapTime", "Driver"])
    
    #Removing variables regarded as unnecessary, adding Qualifying prefix to Data so merging stays clean
    #Note: Qualifying and Practice are merged by driver, so QualifyingDriver is renamed back to driver
    QualiDF >>= select(X.Driver, X.LapTime, X.Compound, X.AirTemp, X.Humidity, X.Pressure, X.Rainfall, X.TrackTemp, X.WindSpeed) \
            >> mutate(LapTime = X.LapTime/ timedelta(seconds = 1))
    QualiDF = QualiDF.add_prefix('Qualifying') >> rename(Driver = X.QualifyingDriver)
    
    return QualiDF

def practiceScrape(grandPrix, raceDate, pracNum, pracDate):
    """
    Input: Grand Prix, Grand Prix Date, Which Practice, Which Date
    Output: Dataframe with each driver's timed lap matched with tire compound and weather conditions
    """
    #Make Path to scrape practice data
    FP = fastf1.api.make_path(grandPrix, raceDate, f'Practice {pracNum}', pracDate)
    
    #Scraping timing data, timing app data (for info on tire compund), and weather data
    FPTiming = fastf1.api.timing_data(FP)[0]
    FPTimingApp = fastf1.api.timing_app_data(FP)
    FPWeatherDict = fastf1.api.weather_data(FP)
    FPWeatherData = pd.DataFrame.from_dict(FPWeatherDict)
    
    #Remove entries with no laptimes, and round session time to nearest minute (for merging purposes)
    FPTiming = FPTiming[FPTiming.LapTime.notnull()]
    FPTimingApp >>= mutate(Time = X.Time.round('60s')) >> arrange(X.Driver, X.Time)
    FPTimingApp = FPTimingApp.reset_index()
    
    #In timing app dataframe, tire compound is only noted when new set is put on
    #Since dataframe is organized by driver and then time, we can replace empty entries with the previous tire compound
    #Also, dataframe sometime doubles on driver-time combo, so removing the unnecessary ones
    for index in range(len(FPTimingApp)):
        if not FPTimingApp.loc[index,"Compound"]:
            FPTimingApp.loc[index, "Compound"] = FPTimingApp.loc[index-1, "Compound"]
        if not FPTimingApp.loc[index,"New"]:
            FPTimingApp.loc[index,"New"] = False
        if index == 0:
            continue
        if FPTimingApp.loc[index,"Time"] == FPTimingApp.loc[index-1,"Time"]:
            if not FPTimingApp.loc[index,"LapTime"]:
                FPTimingApp.loc[index,"Driver"] = np.nan
            else: 
                FPTimingApp.loc[index-1,"Driver"] = np.nan                
    FPTimingApp = FPTimingApp[FPTimingApp.Driver.notnull()]
    
    #Selecting relevant rows, and rounding FPtiming session time to nearest minute
    FPTimingApp >>= select(X.Stint, X.Driver, X.TotalLaps, X.Compound, X.New, X.Time)
    FPTiming = FPTiming >> mutate(Time = X.Time.round('60s')) >> arrange(X.Driver, X.Time) >> \
              drop(contains("Session"), X.PitOutTime, X.PitInTime)
    
    #Merge Dataframes, keyed by time and driver, then dropping irrelevant columns
    FPCompiledTiming = pd.merge(FPTiming,FPTimingApp, on=['Time', 'Driver'])
    FPWeatherData >>= mutate(Time = X.Time.round('60s'))
    FPCompiledTiming = pd.merge(FPCompiledTiming, FPWeatherData, on=['Time'])
    FPCompiledTiming >>= arrange(X.Driver, X.Time) >> mutate(Practice = pracNum) >> drop(X.WindDirection, X.Stint, X.TotalLaps)\
                     >> mutate(LapTime = X.LapTime/timedelta(seconds = 1)) >> mutate(Sector1Time = X.Sector1Time/timedelta(seconds = 1)) \
                     >> mutate(Sector2Time = X.Sector2Time/timedelta(seconds = 1)) >> mutate(Sector3Time = X.Sector3Time/timedelta(seconds = 1))
    FPCompiledTiming = FPCompiledTiming.reset_index()
    return FPCompiledTiming

In [3]:
def weekendCompiler(grandPrix, raceDate, sprint = False):
    """
    Input: Grand Prix with appropriate race date, and whether grand prix was a sprint weekend
    Output: Dataframe with all timed practice laps, merged with info about best qualifying lap
    """
    #Creating relevant dates for session calling purposes
    raceDate = datetime.strptime(raceDate, "%Y-%m-%d")
    friDate = str(raceDate - timedelta(days = 2))[:10]
    satDate = str(raceDate - timedelta(days = 1))[:10]
    raceDate = str(raceDate)[:10]
    
    #Collecting Practice Data
    #Note: On sprint weekends only one practice is done before qualifying, so FP1 is the only relevant data
    FP1CompiledTiming = practiceScrape(grandPrix, raceDate, 1, friDate)
    if not sprint:
        FP2CompiledTiming = practiceScrape(grandPrix, raceDate, 2, friDate)
        FP3CompiledTiming = practiceScrape(grandPrix, raceDate, 3, satDate)
    
    #Collecting Quali Data:
    #Note: On sprint weekends, qualifying is done on Friday, as opposed to usual Saturdays
    if not sprint:
        QualiDF = QualiScrape(grandPrix, raceDate, satDate)
    else:
        QualiDF = QualiScrape(grandPrix, raceDate, friDate)
    
    #Concatenating Practice Dataframes
    if not sprint:
        frames = [FP1CompiledTiming, FP2CompiledTiming, FP3CompiledTiming]
        practiceCompiled = pd.concat(frames)
    else:
        practiceCompiled = FP1CompiledTiming
    
    #Merging Quali Data with compiled Practice data, such that each laptime has the appropriate quali data
    practiceCompiled = practiceCompiled.reset_index()
    practiceCompiled >>= drop(X.level_0)
    practiceCompiled = pd.merge(practiceCompiled, QualiDF, on=['Driver']) >> mutate(Weekend = grandPrix)
    
    return practiceCompiled

Main Code for constructing data frames

In [4]:
raceCalendar = {1:["Bahrain Grand Prix", "2022-03-20"], 2:["Saudi Arabian Grand Prix", "2022-03-27"], 3:["Australian Grand Prix", "2022-04-10"],\
                4:["Emilia Romagna Grand Prix", "2022-04-24"], 5:["Miami Grand Prix", "2022-05-08"], 6:["Spanish Grand Prix", "2022-05-22"], \
                7:["Monaco Grand Prix", "2022-05-29"], 8:["Azerbaijan Grand Prix", "2022-06-12"], 9:["Canadian Grand Prix",  "2022-06-19"], \
                10:["British Grand Prix", "2022-07-03"], 11:["Austrian Grand Prix", "2022-07-10"], 12:["French Grand Prix", "2022-07-24"], \
                13:["Hungarian Grand Prix", "2022-07-31"], 14:["Belgian Grand Prix", "2022-08-28"], 15:["Dutch Grand Prix", "2022-09-04"], \
                16:["Italian Grand Prix", "2022-09-11"], 17:["Singapore Grand Prix", "2022-10-02"], 18:["Japanese Grand Prix", "2022-10-09"], \
                19:["United States Grand Prix", "2022-10-23"], 20:["Mexico City Grand Prix", "2022-10-30"], 21:["São Paulo Grand Prix", "2022-11-13"]}
dfList = []
for key in raceCalendar:
    
    print(f"Collecting Data for {raceCalendar[key][0]}")
    #Imola, Austria, and Brazil are sprint weekends, so they have a special case where sprint = true
    if key not in [4,11,21]:
        dfList.append(weekendCompiler(raceCalendar[key][0], raceCalendar[key][1]))
    else:
        dfList.append(weekendCompiler(raceCalendar[key][0], raceCalendar[key][1], sprint = True))

masterList = pd.concat(dfList)
masterList >>= drop(X.Time)

api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data


...

api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
api            INFO 	Using cached data for weather_data


Index(['index', 'Driver', 'LapTime', 'NumberOfLaps', 'NumberOfPitStops',
       'Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2',
       'SpeedFL', 'SpeedST', 'IsPersonalBest', 'Compound', 'New', 'AirTemp',
       'Humidity', 'Pressure', 'Rainfall', 'TrackTemp', 'WindSpeed',
       'Practice', 'QualifyingLapTime', 'QualifyingCompound',
       'QualifyingAirTemp', 'QualifyingHumidity', 'QualifyingPressure',
       'QualifyingRainfall', 'QualifyingTrackTemp', 'QualifyingWindSpeed',
       'Weekend'],
      dtype='object')

# Neural Network

Clean Data For Neural Network

In [5]:

nn_copy = masterList.copy()
nn_copy = pd.get_dummies(nn_copy, columns = ['Weekend', 'Driver', 'Compound', 'QualifyingCompound'])

#Adjust Boolean Data
nn_copy['New'] = pd.to_numeric(nn_copy['New'], errors = 'coerce')

nn_copy = nn_copy.dropna()


In [6]:
#Set training variables(X) and prediction variables (Y)
x = nn_copy >> drop(X.QualifyingLapTime)
             
y = nn_copy['QualifyingLapTime']

#Normalize Training Data

x_scaler = preprocessing.StandardScaler().fit(x)
x_norm = x_scaler.transform(x)

#Split Data Into Test and Training Sets
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size = 0.2)

x_train = np.asarray(x_train).astype(float)
y_train = np.asarray(y_train).astype(float)
x_test = np.asarray(x_test).astype(float)
y_test = np.asarray(y_test).astype(float)

#Split Test Data Into Validation and Test Sets
x_validation, x_test, y_validation, y_test = train_test_split(x_test, y_test, test_size = 0.5)

"Standard" Neural Network Following General Guidelines 

In [7]:
#One layer is generally enough for simple problems like this (As opposed to something like digit recognition)
#Hidden layer size is generally between the input size (number of x variables) and the output size (number of y variables)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation = 'relu'),
])

#Adam is the recommended default optimizer
#Mean squared error loss function as seen in class
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
#Start Batch Size Small (Powers of 2 for efficient GPU usage)
#Epochs rule of thumb is to start with triple the amount of columns
#Verbose determines how it prints the training progress
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)

#Evaluate Model 
model.evaluate(x_validation, y_validation, verbose = 2)

Epoch 1/100
347/347 - 1s - loss: 7455.8296 - mean_squared_error: 7455.8296 - mean_absolute_error: 85.5617 - 1s/epoch - 4ms/step
Epoch 2/100
347/347 - 1s - loss: 7191.5283 - mean_squared_error: 7191.5283 - mean_absolute_error: 83.9899 - 621ms/epoch - 2ms/step
Epoch 3/100
347/347 - 1s - loss: 6916.8657 - mean_squared_error: 6916.8657 - mean_absolute_error: 82.2843 - 621ms/epoch - 2ms/step
Epoch 4/100
347/347 - 1s - loss: 6653.6943 - mean_squared_error: 6653.6943 - mean_absolute_error: 80.5694 - 624ms/epoch - 2ms/step
Epoch 5/100
347/347 - 1s - loss: 6406.6548 - mean_squared_error: 6406.6548 - mean_absolute_error: 78.8777 - 605ms/epoch - 2ms/step
Epoch 6/100
347/347 - 1s - loss: 6175.5586 - mean_squared_error: 6175.5586 - mean_absolute_error: 77.2164 - 627ms/epoch - 2ms/step
Epoch 7/100
347/347 - 1s - loss: 5959.5942 - mean_squared_error: 5959.5942 - mean_absolute_error: 75.5836 - 605ms/epoch - 2ms/step
Epoch 8/100
347/347 - 1s - loss: 5756.8579 - mean_squared_error: 5756.8579 - mean_abso

KeyboardInterrupt: 

What If We Adjust The Network Architecture?

In [ ]:
#This one is good
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)      
model.evaluate(x_validation, y_validation, verbose = 2)                                                                   

In [ ]:
#This one is bad
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)      
model.evaluate(x_validation, y_validation, verbose = 2)

In [ ]:
#This one is bad
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)      
model.evaluate(x_validation, y_validation, verbose = 2)   

In [ ]:
#This one is good
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)      
model.evaluate(x_validation, y_validation, verbose = 2)   

In [ ]:
#This one is bad
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)      
model.evaluate(x_validation, y_validation, verbose = 2)   

In [ ]:
#This one is best
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)      
model.evaluate(x_validation, y_validation, verbose = 2)   

In [ ]:
#Evaluate best model with test data
model.evaluate(x_test, y_test)

What if we change the activation function? (They are all worse than relu)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'linear'),
    tf.keras.layers.Dense(16, activation = 'linear'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)      
model.evaluate(x_validation, y_validation, verbose = 2)  

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'sigmoid'),
    tf.keras.layers.Dense(16, activation = 'sigmoid'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)      
model.evaluate(x_validation, y_validation, verbose = 2)  

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'tanh'),
    tf.keras.layers.Dense(16, activation = 'tanh'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose = 2)      
model.evaluate(x_validation, y_validation, verbose = 2)  

What if we adjust batch size?

In [ ]:
#This one is bad
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 64, epochs = 100, verbose = 0)
model.evaluate(x_validation, y_validation, verbose = 2)

In [ ]:
#This one is superior
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 8, epochs = 100, verbose = 0)
model.evaluate(x_validation, y_validation, verbose = 2)

In [ ]:
#This one is even more superior (Not going smaller because run time is very long)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
])
model.compile(optimizer = 'adam', loss = tf.keras.losses.MSE, metrics = [tf.keras.metrics.MeanSquaredError(),
                                                                         tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size = 4, epochs = 100, verbose = 0)
model.evaluate(x_validation, y_validation, verbose = 2)

What if we add more epochs?

In [ ]:
#Much better but takes a long time to train and could cause overfitting (Best model with all the data)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
])
model.compile(optimizer='adam', loss=tf.keras.losses.MSE, metrics=[tf.keras.metrics.MeanSquaredError(),
                                                                   tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size=32, epochs=1000, verbose=2)
model.evaluate(x_validation, y_validation, verbose=2)

Final Test

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

# Plotting

In [ ]:
def genModel(layers, batchSize=32, epochs=100, verbose=2):
    """
    Automated neural network generator
    Input: list of layers, batch size, number of epochs, verbosity
    Output: it do the thing
    """
    model = tf.keras.models.Sequential(layers)
    model.compile(optimizer='adam', loss=tf.keras.losses.MSE, metrics=[tf.keras.metrics.MeanSquaredError(),
                                                                   tf.keras.metrics.MeanAbsoluteError()])
    model.fit(x_train, y_train, batchSize, epochs, verbose)
    model.evaluate(x_validation, y_validation, verbose)
    

In [ ]:
%matplotlib inline
class PlotData():
    """
    A module for storing plot data in a more portable, reusable form
    Stores x and y data and metadata (title, labels, etc.)
    Parameters: 2 arrays of x and y datasets (to allow multiple relations per plot), 
                plot type as a string, title, axis labels, and grid boolean
    """
    def __init__(self, xdata, ydata, ptype='plot', title='Figure', xlabel='x axis', ylabel='y axis', grid=True):
        
        # Only check these if we have ydata
        if ydata != None:
            # Throws error if xdata and ydata length do not match
            if not len(xdata) == len(ydata):
                raise Exception('Data mismatch error: must have the same number of corresponding x and y fields')
            
            # Throws error if x and y subdata length do not match
                for i in range(len(xdata)):
                    if not len(xdata[i]) == len(ydata[i]):
                        raise Exception(f'Data mismatch error: {i}th set of x and y data have unequal length')
        
            
        self.xdata = xdata
        self.ydata = ydata
        self.ptype = ptype
        self.title = title
        self.xlabel = xlabel
        self.ylabel = ylabel
        self.grid = grid
        
        
    def plot(self, style='b--'):
        """
        May not be showing it but it do be plotting the data
        Input: Optional style
        """
        # Plots based on type
        if self.ptype == 'plot':
            for i in range(len(self.xdata)):
                plt.plot(self.xdata[i], self.ydata[i], style)
                
        elif self.ptype == 'scatter':
            for i in range(len(self.xdata)):
                plt.scatter(self.xdata[i], self.ydata[i], style)
            
        elif self.ptype == 'bar':
            for i in range(len(self.xdata)):
                plt.bar(self.xdata[i], self.ydata[i])
                
        elif self.ptype == 'boxplot':
            plt.boxplot(self.xdata)
            
        else:
            print(f'{self.title} has invalid plot type')
                
        plt.title(self.title)
        plt.xlabel(self.xlabel)
        plt.ylabel(self.ylabel)
        if self.grid:
            plt.grid()
            
            
    def addData(x, y):
        """
        Clears the figure and adds a new x-y relation to the data
        Input: x and y data to add
        Output: It adds the data wow
        """
        plt.clf()
        self.xdata.append(x)
        self.ydata.append(y)
            
        

In [ ]:
class Plotter():
    """
    Takes plot data and auto-generates subplots
    Parameters: a list of PlotData objects
    """
    
    def __init__(self, plots, subdim=None):
        self.plots = plots
        self.subdim = subdim
        self.arrange(self.subdim)
        self.figsize = (5*self.subdim[1], 5*self.subdim[0])
        
        
        
    def addPlot(plot, loc=-1, subdim=None):
        """
        Adds a new subplot and rearranges the figure
        Input: the PlotData to add, the location to display it, and a custom aspect ratio if desired
        Output:  It does it
        """
        self.plots.insert(loc, plot)
        if not subdim == None:
            self.subdim = subdim
        else:
            arrange()
    
    
    def addData(x, y, loc):
        """
        Adds data to a subplot
        Input: x and y data to add, and which subplot to add to
        Output: It also does it
        """
        self.plots[loc].addData(x, y)
    
    
    def arrange(self, subdim=None):
        """
        Helper function to redistrubte subplots to maximize squareness or fulfill user-set dimensions
        Input: Optional custom dimensions
        Output: :)
        """
        if not subdim == None:
            # Throw error if there are not enough subplot spaces
            if subdim[0] * subdim[1] < len(self.plots):
                raise Exception('Specified plot dimensions cannot fit specidied subplot data. Try doing math')
            self.subdim = subdim
        else:
            # Find the arrangement closest to square that fills all subplots
            for i in range(int(np.sqrt(len(self.plots))), 0, -1):
                if len(self.plots) % i == 0:
                    self.subdim = (i, len(self.plots) // i)
                    break
        
                
    def plot(self):
        """
        Generates the figure
        """
        plt.figure(figsize=self.figsize)
        for i in range(len(self.plots)):
            plt.subplot(self.subdim[0], self.subdim[1], i + 1)
            self.plots[i].plot()
        plt.tight_layout()
        
        

In [ ]:
def genDrivers(weekendDF):
    """
    Generates a dictionary of driver IDs to driver stats
    Input: Dataframe
    Output: Dictionary of driver IDs to driver stat dataframe (which is a subset of the input)
    """
    idList = list(set(weekendDF['Driver'].values))
    return {ID : weekendDF.loc[weekendDF['Driver'] == ID] for ID in idList}


In [ ]:
def getAverages(dataframe, race, driverIDs, category):
    """
    Generates a list of average values for a driver stat in a given race
    Input: Dictionary of races to dictionaries of IDs to driver dataframes, the race to 
           select, the driver to select, and the driver stat to average
    Output: The mean value
    """
    
    averages = []
    for ID in driverIDs:
        # If the driver is not present they get 0
        try:
            averages.append(dataframe[race][ID][category].values.mean().item() / 1.0e9)
        except:
            averages.append(0)
    return averages


In [ ]:
# Trim down to just the data we're interested in
categories = ['Driver', 'LapTime', 'Sector1Time', 'Sector2Time','Sector3Time', 'Compound', 'QualifyingLapTime', 
          'QualifyingSector1Time', 'QualifyingSector2Time','QualifyingSector3Time', 'QualifyingCompound', 'Weekend']
plotDataFrame = masterList.copy()

# Split data into a dictionary by race
raceNames = [race[0] for race in raceCalendar.values()]
raceDataFrames = {race : plotDataFrame.loc[plotDataFrame["Weekend"] == race] for race in raceNames}

# Split dictionary of races further into to dictionaries of driver IDs to driver data
# Get a specific stat with driverDataFrames['raceName']['driverID']['field'].values
driverDataFrames = {name : genDrivers(raceDataFrames[name]) for name in raceNames}

# Make sorted list of driver IDs
driverIDs = list(set([int(ID) for ID in plotDF['Driver'].values]))
driverIDs.sort()
driverIDs = [str(ID) for ID in driverIDs]

# Select races to plot
selectedRaces = raceNames


In [ ]:
def getDataField(field):
    """
    Takes a field name and produces a dictionary of races to dictionaries of drivers to numpy arrays of driver data
    """
    dataField = {}
    buffer = {}
    
    for race in selectedRaces:
        for ID in driverIDs:
            try:
                arr = driverDataFrames[race][ID][field].values
                if arr.dtype == 'timedelta64[ns]':
                    buffer[ID] = arr.astype('float64') / 1e9
                else:
                    buffer[ID] = arr
            except:
                buffer[ID] = np.array([0])
                
        dataField[race] = buffer.copy()
        buffer.clear()
        
    return dataField


In [ ]:
def plotField(field, saveName=None):
    """
    Given a field name, generates boxplots for that field for each driver across each race, optionally saves an image
    """
    lapTimes = getDataField(field)
    fieldPlotData = []

    for race, driverData in lapTimes.items():
        raceData = [driverField for driverField in driverData.values()]
        title = f'{field} of drivers in {race}'
        fieldPlotData.append(PlotData(raceData, None, 'boxplot', title, 'Drivers', field))
        
    plotter = Plotter(fieldPlotData)
    plotter.plot()
    
    if saveName != None:
        plt.savefig(saveName)
    

In [ ]:
# Plots and saves each dataframe field
# FIXME: plots for some races are mostly blank or show negative values?
timeFields = ['LapTime', 'Sector1Time', 'Sector2Time','Sector3Time']
speedFields = ['SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST']
environmentFields = ['AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'Windspeed']

qtimeFields = ['QualifyingLapTime', 'QualifyingSector1Time', 'QualifyingSector2Time','QualifyingSector3Time']
qspeedFields = ['QualifyingSpeedI1', 'QualifyingSpeedI2', 'QualifyingSpeedFL', 'QualifyingSpeedST']

for field in speedFields:
    plotField(field, field + 'BoxPlots.png')
    